**Dados**
<p>Essa é a parte que pode ser alterada para testar outras estratégias de chunking

In [1]:
import pandas as pd
#carrega o csv com os dados pré-processados
data = pd.read_csv("./dataset_preprocessed.csv")

#cria nova lista onde cada elemento é um chunk contendo o nome e a review
new_data = []
for index, row in data.iterrows():
    result = "item " + str(index) + ": " + str(row.product_name) + ". review: " + str(row.review_text)
    new_data.append(result)



**Embedding**

In [ ]:
from sentence_transformers import SentenceTransformer

#carrega o modelo pré-treinado 
model = SentenceTransformer('all-MiniLM-L6-v2')

#transforma cada pedaço de texto em vetores
embeddings = model.encode(new_data)


**BD Vetorial**

In [ ]:
import chromadb
#cria o banco de dados vetorial
client = chromadb.Client()
collection = client.get_or_create_collection("mini_collection")

In [4]:
#salva os vetores no bd criado no passo anterior
collection.add(
    documents=new_data,  # textos
    embeddings=embeddings,  # vetores correspondentes a cada texto
    ids=[str(i) for i in range(len(new_data))]  # cria IDs pra cada registro
    )


**Prompt**

In [5]:
#pergunta do usuário
query = input("Digite a pergunta:")

#vetoriza a pergunta do usuário (query)
query_embedding = model.encode([query])

#faz uma busca semântica no bd vetorizado comparando os vetores da pergunta com os vetores dos dados, recupera 4 resultados 
results = collection.query(query_embeddings=query_embedding, n_results=4)

#seleciona apenas os textos dos resultados 
retrieved_docs = results['documents']

#junta os resultados em um texto só e forma o "contexto" da resposta
context = "\n".join(retrieved_docs[0])

#formar o prompt, passando o contexto (dados recuperados) e a pergunta 
prompt = f"Context: {context}\n\nUser Question: {query}\n\nAnswer:"


**Geração de resposta**

In [6]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

#configurar a api com a chave (cada usuário deve ter uma, salvar como variável de ambiente)
#a chave é obtida em https://ai.google.dev/gemini-api/docs/api-key?hl=pt-br
genai.configure(api_key=os.getenv("API_KEY"))

In [ ]:
#carrega o modelo que vai interpretar o prompt
model = genai.GenerativeModel("gemini-1.5-flash")

#passa o prompt pra gerar a resposta
response = model.generate_content(prompt)

#imprime a resposta gerada
print(response.text)